# Suppa Pig Game Documentation

## Table of Contents
1. [Introduction](#introduction)
2. [Prerequisites](#prerequisites)
3. [Installation](#installation)
4. [How to Play](#how-to-play)
5. [Game Mechanics](#game-mechanics)
6. [Highscore](#highscore)
7. [Customization](#customization)
8. [Known Issues](#known-issues)
9. [Contributing](#contributing)
10. [License](#license)

## Introduction

Suppa Pig is a simple 2D game built using the Pygame library in Python. The game features a pig character that the player controls and obstacles that fall from the top of the screen. The objective of the game is to avoid the obstacles and collect pumpkins to score points. The game keeps track of your current score and your highscore.

## Prerequisites

To run and modify this game, you need to have the following installed:

- Python 3
- Pygame library


## How to Play

- Use the arrow keys (Up, Down, Left, Right) to control the pig character.
- Avoid the falling obstacles (fences) to stay in the game.
- Collect pumpkins to increase your score.
- The game ends when you collide with an obstacle.

## Game Mechanics

- The pig character can move in all directions using the arrow keys.
- Obstacles (fences) fall from the top of the screen at regular intervals.
- The game keeps track of your current score and highscore.
- If you beat your highscore, it will be updated and saved.
- Press 'R' to restart the game after a game over.

## Highscore

- The game stores your highscore in a text file called `highscore.txt`.
- It loads the highscore at the start of the game.
- If your current score surpasses your highscore, it will be updated and saved to the file.



In [10]:
import pygame
import random
import sys
import pygame.mixer

# Initialize Pygame
pygame.init()
pygame.mixer.init()

# Constants
WIDTH, HEIGHT = 800, 600

# Load background image
background_img = pygame.image.load("background.jpg")  # Replace with your background image path
background_img = pygame.transform.scale(background_img, (WIDTH, HEIGHT))

# Initialize the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Suppa Pig")

# Fonts
font = pygame.font.Font(None, 36)

pygame.mixer.music.load("background_music.mp3")
pygame.mixer.music.set_volume(0.5)  # Adjust volume (0.0 to 1.0)
pygame.mixer.music.play(-1)  # -1 means loop the music indefinitely

collect_sound = pygame.mixer.Sound("collect.wav")
game_over_sound = pygame.mixer.Sound("game_over.wav")


# Pig variables
pig_img = pygame.image.load("pig.png")  # Replace with your pig image path
pig_img = pygame.transform.scale(pig_img, (100, 100))
pig_x = WIDTH // 2 - 25
pig_y = HEIGHT - 60
pig_speed = 9

# Obstacle variables
fence_img = pygame.image.load("fence.png")  # Replace with your fence image path
fence_img = pygame.transform.scale(fence_img, (90, 90))
obstacle_x = random.randint(0, WIDTH - 30)
obstacle_y = 0
obstacle_speed = 10
obstacle_spawn_counter = 0
OBSTACLE_SPAWN_DELAY = 10  # Decreased spawn delay for obstacles

# Food variables
pumpkin_img = pygame.image.load("pumpkin.png")  # Replace with your pumpkin image path
pumpkin_img = pygame.transform.scale(pumpkin_img, (70, 70))
food_x = random.randint(0, WIDTH - 20)
food_y = 0
food_speed = 10
food_spawn_counter = 0
FOOD_SPAWN_DELAY = 40

score = 0
highscore = 0
game_over = False
restart = False

# Load and save highscore
try:
    with open("highscore.txt", "r") as file:
        highscore = int(file.read())
except FileNotFoundError:
    with open("highscore.txt", "w") as file:
        file.write(str(highscore))

# Main game loop
while not restart:
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_UP]:
            pig_y -= pig_speed
        if keys[pygame.K_DOWN]:
            pig_y += pig_speed
        if keys[pygame.K_LEFT]:
            pig_x -= pig_speed
        if keys[pygame.K_RIGHT]:
            pig_x += pig_speed

        obstacle_y += obstacle_speed
        food_y += food_speed

        if (
            pig_x < obstacle_x + 30
            and pig_x + 50 > obstacle_x
            and pig_y < obstacle_y + 30
            and pig_y + 50 > obstacle_y
        ):
            game_over = True
        elif (
            pig_x < food_x + 20
            and pig_x + 50 > food_x
            and pig_y < food_y + 20
            and pig_y + 50 > food_y
        ):
            score += 1
            food_x = random.randint(0, WIDTH - 20)
            food_y = 0

        if obstacle_y > HEIGHT:
            obstacle_x = random.randint(0, WIDTH - 30)
            obstacle_y = 0

        if food_y > HEIGHT:
            food_x = random.randint(0, WIDTH - 20)
            food_y = 0

        # Update highscore if necessary
        if score > highscore:
            highscore = score

        # Display background image
        screen.blit(background_img, (0, 0))

        # Draw pig
        screen.blit(pig_img, (pig_x, pig_y))

        # Draw obstacle
        screen.blit(fence_img, (obstacle_x, obstacle_y))

        # Draw food
        screen.blit(pumpkin_img, (food_x, food_y))

        # Display score and highscore
        score_text = font.render(f"Score: {score}", True, (0, 128, 0))
        highscore_text = font.render(f"Highscore: {highscore}", True, (255, 255, 255))
        screen.blit(score_text, (10, 10))
        screen.blit(highscore_text, (10, 50))

        pygame.display.flip()

        pygame.time.delay(30)

    # Game over screen
    screen.fill((0, 0, 0))  # Fill with a black background
    game_over_text = font.render("Game Over", True, (255, 0, 0))
    final_score_text = font.render(f"Final Score: {score}", True, (255, 0, 0))
    restart_text = font.render("Press R to Restart", True, (255, 0, 0))
    screen.blit(game_over_text, (WIDTH // 2 - 100, HEIGHT // 2 - 50))
    screen.blit(final_score_text, (WIDTH // 2 - 70, HEIGHT // 2 + 10))
    screen.blit(restart_text, (WIDTH // 2 - 100, HEIGHT // 2 + 50))
    pygame.display.flip()

    while game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                with open("highscore.txt", "w") as file:
                    file.write(str(highscore))
                pygame.quit()
                sys.exit()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    game_over = False
                    restart = False
                    pig_x = WIDTH // 2 - 25
                    pig_y = HEIGHT - 60
                    score = 0
                    obstacle_x = random.randint(0, WIDTH - 30)
                    obstacle_y = 0
                    food_x = random.randint(0, WIDTH - 20)
                    food_y = 0

pygame.quit()
sys.exit()

SystemExit: 